In [ ]:
import os
from autogen_agentchat.agents import AssistantAgent, CodeExecutorAgent
from autogen_agentchat.teams import MagenticOneGroupChat, SelectorGroupChat, RoundRobinGroupChat
from autogen_ext.agents.file_surfer import FileSurfer
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import TextMentionTermination, SourceMatchTermination
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from dotenv import load_dotenv
from pathlib import Path

In [ ]:
# Get the absolute path to the .env file
env_path = Path(os.getcwd()).parent / ".env"

# Load the .env file
load_dotenv(dotenv_path=env_path)

In [ ]:
# Get environment variable
AZURE_AI_SERVICE_NAME = os.getenv("AZURE_AI_SERVICE_NAME")

# Create the token provider
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o",
    api_version="2024-05-01-preview",
    model="gpt-4o",
    azure_endpoint=f"https://{AZURE_AI_SERVICE_NAME}.openai.azure.com/",
    azure_ad_token_provider=token_provider,
)

In [3]:
def read_file_tool(filePath: str):
    try:
        return file_tool("read", None, filePath)
    except Exception as e:
        return str(e)

def file_tool(operation: str, body: str, filePath: str):
    if operation == "read":
        with open(filePath, "r") as f:
            return f.read()
    elif operation == "write":
        with open(filePath, "w") as f:
            f.write(body)
            return f"File {filePath} written successfully"
    else:
        raise ValueError("Invalid operation")

def directory_write_tool(directory_path: str):
    """Useful to create a directory with the given path."""
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
        return f"Directory '{directory_path}' has been created successfully."
    else:
        return f"Directory '{directory_path}' already exists."

In [ ]:
tools = [file_tool, directory_write_tool]

file_reader = FileSurfer(
    name="file_reader",
    model_client=az_model_client
)

# Create the developer agent.
developer_agent = AssistantAgent(
    name="developer",
    model_client=az_model_client,
    system_message=
    """
    You are a senior dotnet developer.
    Ask the FileSurfer to read the file and provide you with the necessary information.
    Don't assume anything for illustrative purposes. You must be provided with the necessary information.
    You cannot write the file yourself, but you can read it.
    Proceed with implementing the suggested changes after the feedback from the tech-lead. No need to ask for permission.
    """,
    tools=[tool for tool in tools]
)

# Create the developer agent.
tester_agent = AssistantAgent(
    name="tester",
    model_client=az_model_client,
    system_message=
    """
    Write and run test cases for the code implemented by the developer.
    Write and run test cases for the implemented code. 
    Your final answer must include test scripts and test results.
    You have access to tools which can read files, write files and create directories.
    """,
    tools=[tool for tool in tools]
)

# Create the tech-lead agent.
techlead_agent = AssistantAgent(
    name="techlead",
    model_client=az_model_client,
    system_message=
    """
    You are a technical lead.
    You should not write code yourself.
    You must challenge the developer output and provide constructive feedback according to the best practices.
    Don't allow multiple classes in a single file. SPlit them into separate files.
    When your feedbacks are addressed, then you can write the files.
    Once the file is written, respond with 'FINISH' to terminate the conversation.
    """,
    tools=[file_tool]
)

In [ ]:
# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("FINISH")
#text_termination = MaxMessageTermination(5)
#text_termination = SourceMatchTermination("developer")

# Create a team with the primary and critic agents.
team = SelectorGroupChat([file_reader, developer_agent, techlead_agent], model_client=az_model_client, max_turns=5, termination_condition=text_termination)

In [ ]:
task = "The PeopleContract.cs file available in this directory contains a WCF contract. Generate a web api controller that implements the contract."
await Console(team.run_stream(task=task))